# **SIN 393 – Introduction to Computer Vision (2023)**

# Lecture 04 - Part 3 - Artificial Neural Networks

Prof. João Fernando Mari ([*joaofmari.github.io*](https://joaofmari.github.io/))

---

## Importing the required libraries
---

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

from sklearn import datasets, metrics, preprocessing, model_selection

### %matplotlib notebook

In [2]:
def error_brute_force(X, y, w0=[-1, 1], w1=[-1, 1], b=[-1, 1], step=0.1):
    """
    20 x 20 x 20 = 8.000 combinações de parâmtros
    """
    # Initialize error space
    error_space = np.zeros([len(np.arange(w0[0], w0[1], step)),
                            len(np.arange(w1[0], w1[1], step)),
                            len(np.arange(b[0], b[1], step))])

    # Iterate along w0
    for i, w0_ in enumerate(np.arange(w0[0], w0[1], step)):
        # Iterate along w1
        for j, w1_ in enumerate(np.arange(w1[0], w1[1], step)):
            # Iterate along b
            for k, b_ in enumerate(np.arange(b[0], b[1], step)):
                # Initialize epoch error
                erro_epoca = 0.
                w = [w0_, w1_]
                # Iterate along dataset (one epoch)
                for x_, y_ in zip(X, y):
                    # Innet product
                    v = np.dot(x_, w) + b_
                    # Activation function
                    y_out = np.where(v >= 0., 1, 0)

                    # Error
                    erro = y_ - y_out
                    # Update epoch error
                    erro_epoca = erro_epoca + erro**2
            
                # Erro total da época.
                erro_epoca = erro_epoca / 2.
                error_space[i, j, k] = erro_epoca      

    return error_space 

## Training the model with binary functions
---

In [3]:
# Data
X_bin = np.array([[0, 0],
                  [0, 1],
                  [1, 0],
                  [1, 1]])

# Binary functions
# ================
# AND
y_and = np.array([0, 0, 0, 1])

# OR
y_or = np.array([0, 1, 1, 1])

# XOR
y_xor = np.array([0, 1, 1, 0])

In [4]:
# Loading the IRIS dataset
# ========================
iris = datasets.load_iris()

# Selects only the Setosa (0) and Virginica (1) classes.
# Select only 2 attributes: sepal length (0) and sepal width (1)
X_iris = iris.data[iris.target < 2, :2]
y_iris = iris.target[iris.target < 2]

## Select the data

In [5]:
X = X_bin
y = y_and
str_title = 'AND'

## Model

In [6]:
error_space = error_brute_force(X, y)

In [7]:
print(error_space.shape)

(20, 20, 20)


In [8]:
def plot_error(error_space, w0_, w1_, b_, w0=[-1, 1], w1=[-1, 1], b=[-1, 1], step=0.1):

    # Convert index to value.
    w0_v = np.arange(w0[0], w0[1], step)
    w1_v = np.arange(w1[0], w1[1], step)
    b_v = np.arange(b[0], b[1], step)

    w0_2 = w0_v[w0_]
    w1_2 = w1_v[w1_]
    b_2 = b_v[b_]

    # Avoid zero division
    if w1_2 == w0_2 or w1_2 == b_2:
        w1_2 = w1_2 + 0.01

    w = [w0_2, w1_2]

    # Define os pontos extremos do segmento da superficie de decisão plotado.
    # p0 = (x0_min, f_x0_min); p1 = (x0_max, f_x0_max)
    # ----------
    x0_min = X.min() - 1.
    x0_max = X.max() + 1.
    # ----------
    f_x0_min = -(w[0] / w[1]) * x0_min - (b_2 / w[1])
    f_x0_max = -(w[0] / w[1]) * x0_max - (b_2 / w[1])

    colors = ['r', 'g', 'b', 'y', 'c', 'm']

    print(f'w0: {w0_2:.2f} w1: {w1_2:.2f} b: {b_2:.2f} Error: {error_space[w0_,w1_,b_]}')

    fig, ax  = plt.subplots(1, 4, figsize=(12, 3))
    for y_ in np.unique(y):
        ax[0].scatter(X[y==y_][:,0], X[y==y_][:,1], color=colors[y_], label=str(y_))
 
    ax[0].set_xlabel('$x_0$')
    ax[0].set_ylabel('$x_1$')
    ax[0].legend()
    ax[0].set_title(str_title)

    ax[0].set_xlim(X.min()-.5, X.max()+.5)
    ax[0].set_ylim(X.min()-.5, X.max()+.5)

    # Plots the surface decision.
    ax[0].plot([x0_min, x0_max], [f_x0_min, f_x0_max], color='b')
    
    ax[1].plot(w0_v, error_space[:,w1_,b_])
    ax[1].axvline(w0_2, color='r')
    ax[1].set_title('$w_0$')
    ax[1].set_xlabel('$w_0$')
    ax[1].set_ylabel('$Error$')
    
    ax[2].plot(w1_v, error_space[w0_,:,b_])
    ax[2].axvline(w1_2, color='r')
    ax[2].set_title('$w_1$')
    ax[2].set_xlabel('$w_1$')
    ### ax[2].set_ylabel('$Error$')
    
    ax[3].plot(b_v, error_space[w0_,w1_,:])   
    ax[3].axvline(b_2, color='r')
    ax[3].set_title('$b$')
    ax[3].set_xlabel('$b$')
    ### ax[3].set_ylabel('$Error$')

    plt.tight_layout()

In [9]:
slider_w0 = widgets.IntSlider(value=10, min=0, max=20)
slider_w1 = widgets.IntSlider(value=10, min=0, max=20)
slider_b = widgets.IntSlider(value=10, min=0, max=20)

widgets.interact(plot_error, error_space=widgets.fixed(error_space), 
                 w0_=slider_w0, w1_=slider_w1, b_=slider_b, 
                 w0=widgets.fixed([-1, 1]), w1=widgets.fixed([-1, 1]), b=widgets.fixed([-1, 1]), 
                 step=widgets.fixed(0.1))

interactive(children=(IntSlider(value=10, description='w0_', max=20), IntSlider(value=10, description='w1_', m…

<function __main__.plot_error(error_space, w0_, w1_, b_, w0=[-1, 1], w1=[-1, 1], b=[-1, 1], step=0.1)>

## Bibliography
---

* GONZALEZ, R.C.; WOODS, R.E. Digital Image Processing. 3rd ed. Pearson, 2007.